- Using the pre-build RAPIDS image on Google Cloud's AI Platform Notebooks with a T4 GPU, 8vCPUs, 30GB RAM
- https://cloud.google.com/ai-platform/notebooks/docs/images#deciding
- This should provide CUDA 10.0, rapids 0.12

In [1]:
%%bash
nvidia-smi
nvcc --version

Sun Dec  6 06:43:44 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.06    Driver Version: 450.51.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    55W / 400W |      0MiB / 40537MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import numpy as np; print('numpy Version:', np.__version__)
import pandas as pd; print('pandas Version:', pd.__version__)
import xgboost as xgb; print('XGBoost Version:', xgb.__version__)
import cudf; print('cudf Version:', cudf.__version__)
import cuml; print('cudf Version:', cuml.__version__)
import time

numpy Version: 1.18.5
pandas Version: 1.1.4


ModuleNotFoundError: No module named 'xgboost'

Download HIGGs dataset & unzip
https://archive.ics.uci.edu/ml/datasets/HIGGS

In [ ]:
# %%bash
# wget https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz -P /home/jupyter/

## Single Node with CPUs (PANDAS + XGBoost) or single GPU (RAPIDS-cuDF + XGBoost)
- XGBoost w/ RAPIDS examples https://rapids.ai/xgboost.html

### Expected CPUs numbers
[INFO]: ------ Data Ingestion is completed in 104.7611632347107 seconds ---   
TOD0: Add Data transformation steps  
[INFO]: ------ Training is completed in 30.218074321746826 seconds ---

#### Expected GPU numbers
[INFO]: ------ Data Ingestion is completed in 18.212464094161987 seconds ---  
TOD0: Add Data transformation steps  
[INFO]: ------ Training is completed in 5.825598955154419 seconds ---

In [ ]:
def xgboost_fun(gpu_cpu, tree_method):
    colnames = ['label'] + ['feature-%02d' % i for i in range(1, 29)]
    start_time = time.time()
    if gpu_cpu=='cpu':
        df=pd.read_csv('/home/jupyter/HIGGS.csv', header=None, names=colnames)
    else: 
        df=cudf.read_csv('/home/jupyter/HIGGS.csv', header=None, names=colnames)
    print("[INFO]: ------ Data Ingestion is completed in {} seconds ---".format((time.time() - start_time)))

    X = df[df.columns.difference(['label'])]
    y = df['label']
    dtrain=xgb.DMatrix(X,y)
    param =  {
               'max_depth': 8,
               'objective': 'reg:squarederror',
               'tree_method': tree_method
             }

    start_time = time.time()
    bst = xgb.train(param, dtrain)
    print("[INFO]: ------ Training is completed in {} seconds ---".format((time.time() - start_time)))
    return bst

In [ ]:
bst=xgboost_fun('gpu','gpu_hist')
#bst=xgboost_fun('cpu','hist')

## TODO: Single Node with multiple GPUS (Dask + RAPIDS) --- Scales to 4 T4s, 8 V100s, or 16 A100s on GCP


## TODO: Multi-Node with multiple GPUS (Dask + RAPIDS) Scales to 64+ GPUs